In [1]:
import pandas as pd
import numpy as np
from statistics import median, mode
import joblib
import os

In [2]:
def extract_features(row:pd.Series)->pd.Series:
    values = np.array(row['values'])
    features = {
        'feature_median': median(values),
        'feature_mode': mode(values),
        'feature_q_5': np.quantile(values, 0.05, axis=0),
        'feature_q_25': np.quantile(values, 0.25, axis=0),
        'feature_q_75': np.quantile(values, 0.75, axis=0),
        'feature_q_95': np.quantile(values, 0.95, axis=0),
        'feature_std': np.std(values),
        'feature_autocorr': pd.Series(values).autocorr(lag=1),
        'feature_q_75**3': np.quantile(values, 0.75, axis=0)**3,
        'feature_median**2': median(values)**2,
        'feature_autocorr**3': (pd.Series(values).autocorr(lag=1))**3,
        '1/feature_q_5': 1/np.quantile(values, 0.05, axis=0)
    }
    return pd.Series(features)

In [3]:
with open('fitted_model.pkl', 'rb') as fd:
    model = joblib.load(fd) 

In [4]:
data_test = pd.read_parquet('test.parquet')

In [5]:
df_test = data_test.apply(extract_features, axis=1)

C:\Users\aygul\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\aygul\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [6]:
pred_proba = model.predict_proba(df_test)[:, 1]

In [7]:
data_test['score'] = pred_proba

In [8]:
data_test[['id', 'score']].to_csv('submission.csv', sep=',', index=False)